<a href="https://colab.research.google.com/github/XTEP63/Time_Series/blob/main/Proyecto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![iteso](https://upload.wikimedia.org/wikipedia/en/5/5f/Western_Institute_of_Technology_and_Higher_Education_logo.png)

**Integrantes del equipo:**
- Francisco Javier Gonzalez Lazalde
- Maximiliano Aguayo Villanueva
- Esteban Javier Berumen Nieto
- Abdon Islas Leon

**Docente:** Daniel Nuño

**Fecha:** 2 Diciembre 2024

# **Índice de contenidos**
1. [Introducción](Introducción)
2. [Vector autoregression](#Vector-autoregression)
3. [Descarga de los datos](#Descarga-de-los-datos)
3. [Transfomraciones matemáticas](#Transfomraciones-matemáticas)
4. [Ajuste de los modelos](#Ajuste-de-los-modelos)
5. [Validación de resultados y elección del modelo](#Validación-de-resultados-y-elección-del-modelo)
6. [Pronosticos](#Pronosticos)
7. [Conclusión](#Conclusión)

## **Introducción**


En el análisis y pronóstico de series de tiempo, una de las aplicaciones más relevantes es la proyección del desempeño futuro de portafolios de inversión. Este proyecto se centra en explorar y aplicar modelos avanzados de series de tiempo para estimar los precios futuros de las acciones que conforman el índice de Precios y Cotizaciones (IPC). El objetivo principal es proporcionar un marco metodológico sólido para la construcción de un portafolio simulado y el pronóstico de su rendimiento a corto plazo.

El proyecto se estructura en varias etapas clave: recopilación de datos históricos de precios, limpieza y transformación de las series de tiempo, selección y ajuste de modelos avanzados, y validación de resultados. Finalmente, se utiliza una metodología jerárquica "bottoms-up" para consolidar los pronósticos a nivel sectorial y del portafolio total.

Entre los modelos considerados para este análisis se incluyen un modelo simple como línea base, ARIMA para capturar patrones de autocorrelación, ETS para modelar componentes de tendencia y estacionalidad, regresión dinámica para incorporar variables exógenas, y Vector Autoregression (VAR) para modelar relaciones entre múltiples series de tiempo. Este conjunto diverso de modelos permite una comparación exhaustiva de diferentes enfoques, facilitando la selección del más adecuado para cada acción basado en métricas de error cuantitativas.

El análisis culmina con un pronóstico a tres meses, proporcionando una visión anticipada del comportamiento del portafolio en función de las tendencias pasadas y el peso de cada acción en el índice. Los resultados obtenidos son fundamentales no solo para entender el comportamiento de las series de tiempo, sino también para la toma de decisiones estratégicas en la gestión de inversiones.

## **Vector autoregression - Teoría**

El modelo de **Vector Autoregression (VAR)** es una herramienta estadística que analiza y predice cómo varias variables cambian juntas a lo largo del tiempo. A diferencia de otros modelos más simples, el VAR considera cómo cada variable afecta a las demás y a sí misma en diferentes momentos.

### Conceptos básicos:

- **Estructura del modelo**: Cada variable se relaciona con sus valores anteriores y los de otras variables.
- **Igualdad**: Todas las variables se tratan de la misma manera, sin asumir que una causa a la otra.

Un ejemplo simple de VAR con dos variables (Y y X) se ve así:


\begin{aligned}
Y_t &= a_1 + b_{11}Y_{t-1} + b_{12}X_{t-1} + \epsilon_{1t} \\
X_t &= a_2 + b_{21}Y_{t-1} + b_{22}X_{t-1} + \epsilon_{2t}
\end{aligned}


Donde:

- $a_1$ y $a_2$ son puntos de inicio.
- $b_{ij}$ son números que muestran cómo los valores pasados afectan al presente.
- $ϵ_{1t}$ y $ϵ_{2t}$ representa errores o sorpresas en cada predicción.

### Ventajas:

Ayuda a entender cómo diferentes variables se influyen entre sí.
Permite analizar relaciones cambiantes entre múltiples series de datos en el tiempo.

### Desventajas:

Los datos deben tener ciertas características para que el modelo funcione bien.
Puede complicarse si se usan muchas variables o se mira muy atrás en el tiempo.

El VAR se usa mucho en economía y finanzas para entender cómo se relacionan cosas como las tasas de interés, la inflación y los precios de las acciones.

## **Librerias Implentadas**

In [326]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import boxcox, shapiro
from sklearn.preprocessing import PowerTransformer


## **Descarga de los datos**

In [327]:
portafolio = pd.read_excel("ipc_componentes.xlsx")
portafolio.head()

,nombre,ticker,peso,ticker yahoo,segmento
0,Alfa SA A,ALFA A,0.0129,ALFAA.MX,Materials
1,Alsea SA,ALSEA *,0.0113,ALSEA.MX,Consumer Discretionary
2,America Movil SAB de CV B,AMX B,0.0908,AMXB.MX,Communication Services
3,"Arca Continental, SAB de CV",AC *,0.0262,AC.MX,Consumer Staples
4,"Banco del Bajio, S.A.",BBAJIO O,0.0155,BBAJIOO.MX,Financials


In [328]:
ticker = yf.Tickers(portafolio['ticker yahoo'].tolist())
data = ticker.history(start= '2015-01-01', end= '2023-12-31', interval='1mo')['Close']

data.head()

[*********************100%***********************]  35 of 35 completed


Ticker,AC.MX,ALFAA.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,...,MEGACPO.MX,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,65.702629,24.683918,37.795330,11.671396,154.705322,NaN,34.200207,15.430019,11.529756,37.507202,...,38.958515,38.346149,33.040306,262.346069,130.751801,NaN,56.302013,88.730598,23.547867,23.577877
2015-02-01,70.285484,29.104921,43.193317,11.671396,161.849304,NaN,37.858753,16.780148,13.123650,44.206852,...,39.022579,39.573231,34.646221,281.710754,138.726166,NaN,57.645126,92.414062,23.328476,29.800152
2015-03-01,69.477188,27.676182,42.006325,11.124302,162.236099,NaN,38.791325,16.133083,12.534602,41.669647,...,42.517677,41.436157,31.766659,235.338318,126.094269,NaN,56.418179,91.677383,23.035954,31.152863
2015-04-01,69.892448,27.990683,43.400570,11.737049,171.416763,NaN,36.944115,18.360479,13.333281,41.570335,...,45.450417,42.735737,35.035809,230.718765,136.695740,NaN,63.874279,101.554527,23.328476,29.624239
2015-05-01,70.519684,27.218002,43.626663,11.744345,179.291824,NaN,37.374531,16.991688,12.990941,42.587986,...,46.048355,40.197914,37.204628,242.054367,130.000061,NaN,61.689003,105.901932,22.910900,31.618048


## **Pesos y Segmentos**

In [329]:

print(f"{'Ticker':<25} {'Ticker Yahoo':<15} {'Segmento':<30} {'Peso':<10}")
print("=" * 85)


for index, row in portafolio.iterrows():
    print(f"{row['ticker']:<25} {row['ticker yahoo']:<15} {row['segmento']:<30} {row['peso']:<10.4f}")


Ticker                    Ticker Yahoo    Segmento                       Peso      
ALFA A                    ALFAA.MX        Materials                      0.0129    
ALSEA *                   ALSEA.MX        Consumer Discretionary         0.0113    
AMX B                     AMXB.MX         Communication Services         0.0908    
AC *                      AC.MX           Consumer Staples               0.0262    
BBAJIO O                  BBAJIOO.MX      Financials                     0.0155    
CUERVO *                  CUERVO.MX       Materials                      0.0051    
BOLSA A                   BOLSAA.MX       Financials                     0.0049    
CEMEX CPO                 CEMEXCPO.MX     Materials                      0.0649    
KOF UBL                   KOFUBL.MX       Consumer Staples               0.0217    
VESTA *                   VESTA.MX        Industrials                    0.0164    
LIVEPOL C-1               LIVEPOLC-1.MX   Consumer Discretionary         0.0

## **Train/Test**

In [330]:
train = data.loc[:'2022-12-01']
test = data.loc['2023-01-01':]
horizon = len(test)
print(f'Longitud de Train{train.shape}')
print(f'Longitud de Test{test.shape}')

Longitud de Train(96, 35)
Longitud de Test(12, 35)


In [331]:
train.tail()

Ticker,AC.MX,ALFAA.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,...,MEGACPO.MX,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2022-08-01,125.653061,11.680709,35.960003,15.888946,393.241577,31.970020,64.348747,32.173889,7.467389,56.151237,...,37.337002,114.007996,37.010101,165.589996,124.870308,81.147408,100.521370,24.065973,36.832081,60.962524
2022-09-01,132.854172,11.662486,35.556732,16.371164,364.940277,42.173584,68.770172,29.883965,6.957568,58.362064,...,35.852360,109.637718,32.999531,194.630005,120.814087,76.467590,104.911148,20.596834,37.288517,65.490273
2022-10-01,148.291702,11.990494,37.435387,17.357035,428.500702,46.135365,74.639496,31.758345,7.717302,78.347603,...,36.470959,136.733429,32.573826,222.009995,133.813965,71.484375,127.627113,19.904902,42.835152,70.869858
2022-11-01,147.137329,12.300278,38.163239,18.579897,440.969513,51.850597,79.556511,33.676933,8.816917,81.575806,...,43.302086,144.946091,36.942413,257.119995,153.584396,74.683937,130.187042,20.141865,42.755775,70.601349
2022-12-01,147.468262,11.325367,36.176388,17.252762,418.400360,50.767097,80.576439,33.358639,7.917233,81.213852,...,45.838348,129.654449,33.766148,238.179993,142.352448,79.271805,130.307175,16.776989,45.821808,64.403969


In [332]:
test.head()

Ticker,AC.MX,ALFAA.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,...,MEGACPO.MX,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-01,154.989120,12.418725,44.281162,18.105921,470.210815,62.561474,91.532082,36.152527,10.016499,90.741959,...,52.324825,148.883667,37.304504,266.190002,175.657791,95.637360,150.583359,21.838522,50.862404,68.770508
2023-02-01,144.603638,10.933581,39.835339,17.641422,482.771729,57.822227,85.403191,31.537308,9.096820,90.976738,...,41.561165,151.393341,37.791683,218.910004,170.572922,104.638420,140.288177,17.326744,52.578987,67.414726
2023-03-01,152.451080,10.432458,42.402508,18.485102,507.967346,54.216091,88.683037,34.065952,9.916533,100.759186,...,40.739307,177.926193,38.964912,268.200012,171.815460,105.778488,127.211227,18.094503,56.260208,67.321220
2023-04-01,160.261185,10.479136,47.635197,18.485102,474.671173,48.897865,94.479050,35.029667,10.806223,103.987389,...,43.231384,174.389587,41.162216,278.000000,178.840622,109.768738,121.647766,17.345699,56.359436,67.760689
2023-05-01,170.277557,10.811080,50.605637,18.485102,457.391785,46.019569,92.853813,32.881203,10.616288,90.608253,...,36.320728,161.719727,37.203983,269.970001,164.580017,109.915848,127.349869,16.075577,55.657761,63.646519


## **Transfomraciones matemáticas**

In [333]:
def aplicar_boxcox(dataframe):
    resultado = dataframe.copy()
    lambdas = {}

    for columna in resultado.columns:
        serie = resultado[columna].dropna()

        if (serie <= 0).any():
            print(f"La columna {columna} tiene valores no positivos y no puede transformarse con Box-Cox.")
            continue

        stat, p_value = shapiro(serie)
        if p_value < 0.05:
            transformed_data, lambda_val = boxcox(serie)
            resultado.loc[serie.index, columna] = transformed_data
            lambdas[columna] = lambda_val
        else:
            lambdas[columna] = None

    return resultado, lambdas

In [334]:
data_boxcox, lambdas = aplicar_boxcox(data)

print("DataFrame transformado:")
display(data_boxcox)
print("\nLambdas aplicadas:")
print(lambdas)

DataFrame transformado:


Ticker,AC.MX,ALFAA.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,...,MEGACPO.MX,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,0.632167,3.273597,122.749534,1.473715,3.858890,NaN,0.555395,24.433214,6.906465,0.833414,...,38.958515,1.934555,3.296954,2.316007,130.751801,NaN,4.541749,13.503228,0.738085,4.448162
2015-02-01,0.632253,3.445515,148.577668,1.473715,3.884591,NaN,0.555557,27.258513,7.737476,0.835462,...,39.022579,1.941993,3.339038,2.324486,138.726166,NaN,4.571565,13.780051,0.737946,4.907150
2015-03-01,0.632239,3.392953,142.774085,1.457991,3.885947,NaN,0.555592,25.896604,7.433590,0.834771,...,42.517677,1.952692,3.262044,2.302622,126.094269,NaN,4.544354,13.725201,0.737756,4.996665
2015-04-01,0.632246,3.404749,149.597981,1.475530,3.917103,NaN,0.555521,30.642919,7.844764,0.834742,...,45.450417,1.959766,3.348943,2.300120,136.695740,NaN,4.701815,14.440697,0.737946,4.895273
2015-05-01,0.632257,3.375528,150.713401,1.475730,3.942385,NaN,0.555538,27.706814,7.669327,0.835032,...,46.048355,1.945657,3.402084,2.306142,130.000061,NaN,4.657541,14.742932,0.737673,5.026735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.632806,2.461485,233.262200,1.614235,4.417754,3.733801,0.556169,62.596531,7.973144,0.841111,...,39.013447,2.213298,3.415959,2.305754,157.880508,9.664303,5.513897,5.040874,0.746158,6.552913
2023-09-01,0.632790,2.467863,252.168435,1.542866,4.375188,3.749850,0.556169,59.113499,6.808822,0.841400,...,36.394905,2.207773,3.373732,2.284887,150.550323,9.587930,5.506943,3.971559,0.745759,6.502243
2023-10-01,0.632797,2.453227,231.164127,1.540129,4.328366,3.754628,0.556114,48.721498,6.508731,0.841522,...,33.417137,2.165060,3.185949,2.281167,144.053970,10.035603,5.625193,3.327076,0.745694,6.460060



Lambdas aplicadas:
{'AC.MX': np.float64(-1.5797335801893004), 'ALFAA.MX': np.float64(0.012941449389984531), 'ALSEA.MX': np.float64(1.4230074722788801), 'AMXB.MX': np.float64(-0.4587479600327536), 'ASURB.MX': np.float64(-0.1112355232724147), 'BBAJIOO.MX': np.float64(-0.017534435700050647), 'BIMBOA.MX': np.float64(-1.7973727939322652), 'BOLSAA.MX': np.float64(1.2657199599506046), 'CEMEXCPO.MX': np.float64(0.740612922207427), 'CHDRAUIB.MX': np.float64(-1.1834288249107427), 'CUERVO.MX': np.float64(0.3338840604490008), 'FEMSAUBD.MX': np.float64(-0.00923415124534175), 'GAPB.MX': np.float64(0.04563283865823975), 'GCARSOA1.MX': np.float64(-0.9161847494470223), 'GCC.MX': np.float64(0.8592712030931149), 'GENTERA.MX': None, 'GFINBURO.MX': None, 'GFNORTEO.MX': np.float64(-0.555397035327299), 'GMEXICOB.MX': np.float64(-0.5279212576450699), 'GRUMAB.MX': np.float64(-0.13123452307360545), 'KIMBERA.MX': np.float64(-0.33669097085344213), 'KOFUBL.MX': np.float64(-0.8464405881756375), 'LABB.MX': np.float

## **Diferenciacion**

In [335]:
from statsmodels.tsa.stattools import adfuller
import pandas as pd

def aplicar_diferenciacion(dataframe, significancia=0.05):
    resultado = dataframe.copy()
    diferenciaciones = {}

    for columna in resultado.columns:
        serie = resultado[columna].dropna()
        
        adf_stat, p_value, *_ = adfuller(serie)
        
        diferencia_nivel = 0
        while p_value >= significancia:
            diferencia_nivel += 1
            serie = serie.diff().dropna()  
            adf_stat, p_value, *_ = adfuller(serie)
        
        resultado.loc[serie.index, columna] = serie
        diferenciaciones[columna] = diferencia_nivel

    return resultado, diferenciaciones

In [336]:
data_diferenciada, niveles_diferenciacion = aplicar_diferenciacion(data_boxcox)
data_diff_cols = data_diferenciada.copy()

print("DataFrame diferenciado:")
display(data_diferenciada)
print("\nNiveles de diferenciación aplicados:")
print(niveles_diferenciacion)


DataFrame diferenciado:


Ticker,AC.MX,ALFAA.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,...,MEGACPO.MX,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,0.632167,3.273597,122.749534,1.473715,3.858890,NaN,5.553945e-01,24.433214,6.906465,0.833414,...,38.958515,1.934555,3.296954,2.316007,130.751801,NaN,4.541749,13.503228,0.738085,4.448162
2015-02-01,0.000086,0.171918,25.828135,0.000000,0.025701,NaN,1.624578e-04,2.825298,7.737476,0.835462,...,0.064064,0.007438,0.042084,0.008479,138.726166,NaN,0.029816,0.276823,-0.000139,0.458989
2015-03-01,-0.000014,-0.052562,-5.803584,-0.015724,0.001355,NaN,3.469004e-05,-1.361908,7.433590,-0.002739,...,3.495098,0.010698,-0.076993,-0.021863,126.094269,NaN,-0.027211,-0.054849,-0.000190,0.089515
2015-04-01,0.000007,0.011797,6.823897,0.017539,0.031156,NaN,-7.111588e-05,4.746314,7.844764,0.000662,...,2.932739,0.007074,0.086899,-0.002502,136.695740,NaN,0.157462,0.715496,0.000190,-0.101392
2015-05-01,0.000011,-0.029221,1.115420,0.000201,0.025282,NaN,1.745217e-05,-2.936105,7.669327,0.000318,...,0.597939,-0.014108,0.053141,0.006022,130.000061,NaN,-0.044275,0.302235,-0.000273,0.131461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,-0.000003,0.093641,10.652012,0.006228,-0.011822,0.051743,-1.269115e-05,-3.262195,7.973144,-0.000143,...,-1.124935,0.005999,-0.001860,0.000788,157.880508,0.286769,-0.043565,-0.489170,0.000175,-0.086972
2023-09-01,-0.000017,0.006378,18.906235,-0.071369,-0.042566,0.016049,-4.688357e-07,-3.483032,6.808822,0.000407,...,-2.618542,-0.005525,-0.042227,-0.020866,150.550323,-0.076372,-0.006954,-1.069315,-0.000399,-0.050670
2023-10-01,0.000007,-0.014636,-21.004308,-0.002737,-0.046822,0.004778,-5.478305e-05,-10.392001,6.508731,-0.000168,...,-2.977768,-0.042713,-0.187783,-0.003720,144.053970,0.447673,0.118250,-0.644483,-0.000065,-0.042183



Niveles de diferenciación aplicados:
{'AC.MX': 1, 'ALFAA.MX': 1, 'ALSEA.MX': 1, 'AMXB.MX': 1, 'ASURB.MX': 1, 'BBAJIOO.MX': 1, 'BIMBOA.MX': 1, 'BOLSAA.MX': 1, 'CEMEXCPO.MX': 0, 'CHDRAUIB.MX': 2, 'CUERVO.MX': 1, 'FEMSAUBD.MX': 1, 'GAPB.MX': 1, 'GCARSOA1.MX': 1, 'GCC.MX': 1, 'GENTERA.MX': 1, 'GFINBURO.MX': 1, 'GFNORTEO.MX': 1, 'GMEXICOB.MX': 1, 'GRUMAB.MX': 1, 'KIMBERA.MX': 1, 'KOFUBL.MX': 1, 'LABB.MX': 0, 'LACOMERUBC.MX': 1, 'LIVEPOLC-1.MX': 1, 'MEGACPO.MX': 1, 'OMAB.MX': 1, 'ORBIA.MX': 1, 'PE&OLES.MX': 1, 'PINFRA.MX': 0, 'Q.MX': 1, 'RA.MX': 1, 'TLEVISACPO.MX': 1, 'VESTA.MX': 1, 'WALMEX.MX': 1}


## **Formato y Elimanación de Na**

### **Datos sin Boxcox ni Diferenciación**

In [337]:
data.index = data.index.strftime('%Y-%m')
data.index = pd.to_datetime(data.index)
data = data.melt(ignore_index=False)
df = pd.DataFrame({"ds": data.index,
                   "y": data['value'],
                   "unique_id": data['Ticker']})
df.reset_index(drop=True, inplace=True)
df.dropna(inplace=True)
df.head()

,ds,y,unique_id
0,2015-01-01,65.702629,AC.MX
1,2015-02-01,70.285484,AC.MX
2,2015-03-01,69.477188,AC.MX
3,2015-04-01,69.892448,AC.MX
4,2015-05-01,70.519684,AC.MX


### **Datos con Boxcox**

In [338]:
data_boxcox.index = data_boxcox.index.strftime('%Y-%m')
data_boxcox.index = pd.to_datetime(data_boxcox.index)
data_boxcox = data_boxcox.melt(ignore_index=False)
df_boxcox = pd.DataFrame({"ds": data_boxcox.index,
                   "y": data_boxcox['value'],
                   "unique_id": data_boxcox['Ticker']})
df_boxcox.reset_index(drop=True, inplace=True)
df_boxcox.dropna(inplace=True)
df_boxcox.head()

,ds,y,unique_id
0,2015-01-01,0.632167,AC.MX
1,2015-02-01,0.632253,AC.MX
2,2015-03-01,0.632239,AC.MX
3,2015-04-01,0.632246,AC.MX
4,2015-05-01,0.632257,AC.MX


### **Datos con Boxcox y Diferenciación**

In [339]:
data_diferenciada.index = data_diferenciada.index.strftime('%Y-%m')
data_diferenciada.index = pd.to_datetime(data_diferenciada.index)
data_diferenciada = data_diferenciada.melt(ignore_index=False)
df_diff = pd.DataFrame({"ds": data_diferenciada.index,
                   "y": data_diferenciada['value'],
                   "unique_id": data_diferenciada['Ticker']})
df_diff.reset_index(drop=True, inplace=True)
df_diff.dropna(inplace=True)
df_diff.head()

,ds,y,unique_id
0,2015-01-01,0.632167,AC.MX
1,2015-02-01,0.000086,AC.MX
2,2015-03-01,-0.000014,AC.MX
3,2015-04-01,0.000007,AC.MX
4,2015-05-01,0.000011,AC.MX


## **Modelos**

### **Modelo simple** 

In [340]:
from statsforecast import StatsForecast
from statsforecast.models import SeasonalNaive
from sklearn.metrics import mean_squared_error

In [341]:
def revertir_boxcox(predicciones, lambda_val):
    if lambda_val is None:
        return predicciones  
    
    if lambda_val == 0:
        return np.exp(predicciones)  
    
    return (predicciones * lambda_val + 1) ** (1 / lambda_val)


In [342]:
model = SeasonalNaive(season_length=12)  

In [343]:
resultados_modelos = {}

In [344]:
todas_predicciones_revertidas = pd.DataFrame()

In [345]:
pesos = portafolio.set_index("ticker yahoo")["peso"].to_dict()

In [346]:
tickers = df_boxcox["unique_id"].unique()

In [347]:
pred = pd.DataFrame(columns=['unique_id', 'pred'] )

In [348]:
for ticker in tickers:
    df_ticker = df_boxcox[df_boxcox["unique_id"] == ticker].copy()
    
    train = df_ticker[df_ticker["ds"] <= '2022-12-31']
    test = df_ticker[df_ticker["ds"] >= '2023-01-01']
    
    sf = StatsForecast(df=train, models=[model], freq='M')
    
    predicciones = sf.forecast(h=len(test))
    
    lambda_val = lambdas.get(ticker, None)
    predicciones_revertidas = revertir_boxcox(predicciones["SeasonalNaive"], lambda_val)

    predicciones_df = pd.DataFrame({
        'unique_id': [ticker] * len(test),  
        'pred': predicciones_revertidas,    
    })
    
    pred = pd.concat([pred, predicciones_df], axis=0, ignore_index=True)
    
    mse = mean_squared_error(test["y"], predicciones_revertidas)
    
    resultados_modelos[ticker] = {"SeasonalNaive_MSE": mse}

c:\Users\esteb\.conda\envs\ITESO_1\lib\site-packages\statsforecast\core.py:476: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
c:\Users\esteb\.conda\envs\ITESO_1\lib\site-packages\statsforecast\core.py:476: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
c:\Users\esteb\.conda\envs\ITESO_1\lib\site-packages\utilsforecast\processing.py:384: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
c:\Users\esteb\.conda\envs\ITESO_1\lib\site-pac

In [349]:
resultados_df = pd.DataFrame(resultados_modelos).T
resultados_df

,SeasonalNaive_MSE
AC.MX,16481.689557
ALFAA.MX,105.451203
ALSEA.MX,28419.712526
AMXB.MX,252.199800
ASURB.MX,149656.495351
BBAJIOO.MX,1270.103288
BIMBOA.MX,4473.739684
BOLSAA.MX,1031.029165
CEMEXCPO.MX,8.977106
CHDRAUIB.MX,3556.692678


In [350]:
pred

,unique_id,pred
0,AC.MX,108.967514
1,AC.MX,120.676102
2,AC.MX,120.792191
3,AC.MX,115.647934
4,AC.MX,122.574722
...,...,...
415,WALMEX.MX,60.962524
416,WALMEX.MX,65.490273
417,WALMEX.MX,70.869858
418,WALMEX.MX,70.601349


### **Vector Autoregresivo**

In [351]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error

In [352]:
def preprocesar_datos(data, umbral_nulos=20):
    data_filtrada = data.loc[:, data.isnull().sum() <= umbral_nulos]
    
    data_imputada = data_filtrada.interpolate(method='linear').dropna()
    
    return data_imputada

In [353]:
data_preprocesada = preprocesar_datos(data_diff_cols)

In [354]:
def dividir_datos(data):
    train = data.loc[:'2022-12-31']
    test = data.loc['2023-01-01':]
    return train, test

In [355]:
train_data, test_data = dividir_datos(data_preprocesada)

In [356]:
model = VAR(train_data)
results = model.fit(ic='aic')

In [357]:
def calcular_mse_por_accion(results, train, test):
   
    forecast = results.forecast(train.values, steps=len(test))
    forecast_df = pd.DataFrame(forecast, index=test.index, columns=test.columns)
    
    mse_por_accion = {}
    for columna in test.columns:
        mse = mean_squared_error(test[columna], forecast_df[columna])
        mse_por_accion[columna] = mse
    
    return mse_por_accion, forecast_df

In [358]:
mse_resultados, forecast_df = calcular_mse_por_accion(results, train_data, test_data)

In [359]:
mse_df = pd.DataFrame.from_dict(mse_resultados, orient='index', columns=['VAR_MSE'])
mse_df.index.name = 'Acción'

In [360]:
print("Métricas de error cuadrático medio (MSE) por acción:")
mse_df

Métricas de error cuadrático medio (MSE) por acción:


,VAR_MSE
Acción,
AC.MX,2.053999e-10
ALFAA.MX,4.647353e-03
ALSEA.MX,4.497093e+02
AMXB.MX,4.993082e-04
ASURB.MX,2.105531e-03
BIMBOA.MX,7.823944e-10
BOLSAA.MX,5.626685e+01
CEMEXCPO.MX,2.803585e+00
CHDRAUIB.MX,3.583625e-07


In [361]:
def revertir_diferenciacion(predicciones, data_original, niveles_diferenciacion):
    
    predicciones_revertidas = predicciones.copy()
    for columna, nivel in niveles_diferenciacion.items():
        if columna not in data_original.columns:
            continue  
        
        if nivel == 0:
            continue  
        
        valores_base = data_original[columna].iloc[-nivel:]
        for _ in range(nivel):
            predicciones_revertidas[columna] = predicciones_revertidas[columna].cumsum() + valores_base.iloc[_]
    
    return predicciones_revertidas


In [362]:
def revertir_boxcox(predicciones, lambdas):

    predicciones_revertidas = predicciones.copy()
    for columna, lambda_val in lambdas.items():
        if columna not in predicciones.columns:
            continue 
        
        if lambda_val is None:
            continue  
        
        if lambda_val == 0:
            predicciones_revertidas[columna] = np.exp(predicciones_revertidas[columna])
        else:
            predicciones_revertidas[columna] = (predicciones_revertidas[columna] * lambda_val + 1) ** (1 / lambda_val)
    
    return predicciones_revertidas


In [363]:
predicciones_revertidas_dif = revertir_diferenciacion(forecast_df, train_data, niveles_diferenciacion)

In [364]:
predicciones_finales = revertir_boxcox(predicciones_revertidas_dif, lambdas)

In [365]:
print("Predicciones en la escala original:")
predicciones_finales

Predicciones en la escala original:


Ticker,AC.MX,ALFAA.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,FEMSAUBD.MX,...,MEGACPO.MX,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-01,1.000015,0.962234,NaN,0.999512,0.961116,1.000040,NaN,8.111734,0.997791,0.972590,...,-1.363354,0.981754,0.970275,1.006583,141.715424,1.192827,0.996790,0.344488,1.000587,0.828913
2023-02-01,1.000000,0.956473,NaN,1.003748,0.966282,1.000011,3.268835,8.103678,0.994661,0.939957,...,-0.840145,0.984508,0.955667,0.998495,139.743233,1.357292,0.937772,0.222197,1.000746,0.818877
2023-03-01,1.000002,0.939073,NaN,0.999293,0.967171,1.000019,3.400328,8.200308,0.992682,0.923204,...,-1.519175,0.983348,0.950513,1.000286,140.425504,1.362650,0.952366,0.145332,1.000828,0.815132
2023-04-01,1.000002,0.916534,NaN,0.995421,0.969642,1.000043,3.263389,8.034479,0.990103,0.915949,...,-1.506561,0.985787,0.940651,1.000905,139.730598,1.414386,0.925672,0.109445,1.000878,0.807013
2023-05-01,1.000012,0.935532,NaN,0.998122,0.975792,1.000055,4.603226,8.280321,0.987767,0.918259,...,-1.561071,0.989383,0.953739,1.005109,141.194061,1.557664,0.959558,0.106571,1.001261,0.840782
2023-06-01,1.000015,0.925328,NaN,0.998172,0.981441,1.000056,5.297954,8.381098,0.985203,0.912562,...,-2.208671,0.993253,0.953851,1.006732,142.000259,1.602091,0.978940,0.077754,1.001326,0.828462
2023-07-01,1.000017,0.908120,NaN,0.998172,0.984964,1.000065,5.647731,8.512115,0.982827,0.907766,...,-2.314469,0.994644,0.951892,1.008708,141.224192,1.652442,0.985975,0.061414,1.001518,0.841800
2023-08-01,1.000019,0.900001,NaN,0.998767,0.988430,1.000079,6.075596,8.646404,0.980402,0.908013,...,-2.511497,0.996702,0.952040,1.009997,141.605709,1.743344,0.999463,0.045882,1.001701,0.848837
2023-09-01,1.000023,0.894971,NaN,0.999753,0.992133,1.000086,6.656155,8.777759,0.977981,0.906324,...,-2.864663,0.999059,0.956652,1.012067,141.872307,1.833076,1.015706,0.033569,1.001868,0.856567


## **Validación de resultados y elección del modelo**

## **Pronosticos**

## **Conclusión**